In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import trapz
from scipy.integrate import quad
import pandas as pd
from scipy import stats
from scipy import optimize

import scipy.io as sio
from statsmodels.stats.weightstats import DescrStatsW
import math
import sklearn.metrics

In [ ]:
def normal(x,mu,sigma):
  d=1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x - mu)**2 / (2 * sigma**2) )

  return d


def triad_trpz(f, a, b,p0,p1, n=1000):
  h = (b - a) / n
  s=np.linspace(a,b,n)
  integ1= (f(a,p0,p1)+f(b,p0,p1))/2.0
  integ2=f(s,p0,p1)
  integ2=sum(integ2)

  integral=(integ1+integ2)*h
  return integral
def pantad_trpz(f, a, b,p0,p1,p2,p3 ,n=1000):
  h = float(b - a) / n
  s=np.linspace(a,b,n)
  integ1= (f(a,p0,p1,p2,p3)/2.0)+(f(b,p0,p1,p2,p3)/2.0)
  integ_2=[f(x,p0,p1,p2,p3) for x in s]
  integ_2=sum(integ_2)
  integral=((integ1+integ_2)*h)
  return (integral)
def trpz(f, a, b, n=1000):
  h = float(b - a) / n
  s=np.linspace(a+h,b-h,n-1)
  integ1= (f(a)/2.0)+(f(b)/2.0)
  integ2=f(s)
  integ2=np.sum(integ2)
  integral=(integ1+integ2)*h
  return integral

In [ ]:
def p1_new(t_m,t_s,w_m):
  return normal(t_m,t_s,t_s*w_m)
  
def p1_new_modified(t_m,t_s,w_m):
  return t_m * normal(t_m,t_s,t_s*w_m)

def p2_new(t_e,t_p,w_p):
  return normal(t_p,t_e,t_e*w_p)

def f_bls(ts,wm,tmin=0.4,tmax=1):
  y1_integ=triad_trpz(p1_new_modified,tmin,tmax,ts,wm)
  y2_integ=triad_trpz(p1_new,tmin,tmax,ts,wm)+(10**(-52))
  return (y1_integ/y2_integ)

def f_mle(tm,wm):
  return tm*((-1+np.sqrt(1+4*wm**2))/(2*wm**2))

def combined_mle(tm,tp,ts,wp,wm):
  return p1_new(tm,ts,wm)*p2_new(f_mle(ts,wm),tp,wp)

def combined_bls(tm,tp,ts,wp,wm):
  return (p1_new(tm,ts,wm)*p2_new(f_bls(ts,wm),tp,wp))

def bayesian(tp,ts,wp,wm):
  return pantad_trpz(combined_bls,-50,+50,tp,ts,wp,wm)

def bayesian_mle(tp,ts,wp,wm):
  return pantad_trpz(combined_mle,-50,+50,tp,ts,wp,wm)

def distractor(ts,td,k):
  return (ts+td*(k**(abs(ts-td))))/(1+(k**(abs(ts-td))))

def bayesian_new(tp,ts,td,wp,wm,k):
  return pantad_trpz(combined_bls,-50,+50,tp,distractor(ts,td,k),wp,wm)


In [ ]:
def reader(dir):
  df=pd.read_csv(dir)
  df=df.dropna()
  df_500=df.query('random_gaps==0.5')
  df_900=df.query('random_gaps==0.9')
  df_1700=df.query('random_gaps==1.7')
  l_tp500=vectorize_data(df_500)
  l_tp900=vectorize_data(df_900)
  l_tp1700=vectorize_data(df_1700)
  return [[l_tp500,l_tp900,l_tp1700],len(df)]

  
def vectorize_data(df):
  #l_tp=np.array(df['t_prod'])
  t_p400=np.array(df.query('t_sample==0.4')['t_prod'])
  t_p600=np.array(df.query('t_sample==0.6')['t_prod'])
  t_p1000=np.array(df.query('t_sample==1')['t_prod'])
  l_tp=[t_p400,t_p600,t_p1000]
  return l_tp

def fitter_plotter(dir):
  df=pd.read_csv(dir)
  df=df.dropna()
  df_500=df.query('random_gaps==0.5')
  df_900=df.query('random_gaps==0.9')
  df_1700=df.query('random_gaps==1.7')
  l_tp500=vectorize_data(df_500)
  l_tp900=vectorize_data(df_900)
  l_tp1700=vectorize_data(df_1700)
  bnds = ((0, 1), (0, 1),(0,1))
  minimum= optimize.minimize(log_like_2,[0.3,0.3,0.3],args=(l_tp500,l_tp900,l_tp1700),bounds=bnds,method='Nelder-Mead')
  print('our fitted parameters ',minimum)



